In [30]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)


# the laptops information:

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_manifacture(soup):
    
    try:
        # Outer Tag Object
        man = soup.find("span", attrs={"class":'a-size-base po-break-word'})

            # Inner NavigatableString Object
        man_value = man.text

            # Title as a string value
        man_string = man_value.strip()
    except:
        man_string = ""

    return man_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={"class":'a-price-whole'}).text
    except:
        price = ""
    return price



# get discount
def get_discount(soup):
    try:
        discount = soup.find("span", attrs={"class":'a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage'}).text
    except:
        discount = ''
    return discount




# Function to extract Availability Status
def get_availability(soup):
    try:    
        available = soup.find("span", attrs={"class":'a-size-medium a-color-success'}).text
    except :
        available = "Not Available"

    return available
# Function to extract Product Rating


def get_rating(soup):

    try:
        rating = soup.find("span", attrs={"class":'a-icon-alt'}).text
    except:
        rating = "no rating"

    return rating
# Function to extract Product description
def get_desc(soup):

    try:
        desc = soup.find("div", attrs={"class":'a-section a-spacing-small a-spacing-top-small'}).text
    except:
        desc = "no description"
    return desc


def get_nrate(soup):
    try:
        
        nrate = soup.find("span", attrs={"id":"acrCustomerReviewText","class":'a-size-base'}).text
        
    except:
        
        nrate = "0 rates"

    return nrate




In [84]:
def data_generate(URL):
    # add your user agent 
    HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5' ,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers'}
    # The webpage URL
#     URL = 'https://www.amazon.eg/deal/912cee26?pf_rd_r=JTWKC8K5YSK39E4EDRJE&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=ee55a2a2-1309-4ad7-bf63-ee00919ab1d3&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_tlt_0_912cee26_dt_sl8_d3&pageNum=1&sortOption=price-asc-rank&language=en_AE'
    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-size-base a-color-base a-link-normal a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[],"link":[], "manifacture":[],"price":[],"discount":[], "availability":[] ,"rating":[],"number_of_rating":[] , "description":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.eg" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['link'].append("https://www.amazon.eg" + link)
        d['manifacture'].append(get_manifacture(new_soup))
        d['price'].append(get_price(new_soup))
        d['discount'].append(get_discount(new_soup))
        d['availability'].append(get_availability(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['number_of_rating'].append(get_nrate(new_soup))
        d['description'].append(get_desc(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    return amazon_df

In [87]:
amazon_df1 = data_generate('https://www.amazon.eg/deal/912cee26?pf_rd_r=JTWKC8K5YSK39E4EDRJE&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=ee55a2a2-1309-4ad7-bf63-ee00919ab1d3&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_tlt_0_912cee26_dt_sl8_d3&pageNum=1&sortOption=price-asc-rank&language=en_AE')

In [91]:
amazon_df2 = data_generate('https://www.amazon.eg/deal/912cee26?pf_rd_r=JTWKC8K5YSK39E4EDRJE&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=ee55a2a2-1309-4ad7-bf63-ee00919ab1d3&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_tlt_0_912cee26_dt_sl8_d3&language=en_AE&pageNum=2&sortOption=price-asc-rank')

In [85]:
amazon_df3 = data_generate('https://www.amazon.eg/deal/912cee26?pf_rd_r=JTWKC8K5YSK39E4EDRJE&pf_rd_t=Events&pf_rd_i=blackfriday&pf_rd_p=ee55a2a2-1309-4ad7-bf63-ee00919ab1d3&pf_rd_s=slot-8&ref=dlx_black_gd_dcl_tlt_0_912cee26_dt_sl8_d3&language=en_AE&pageNum=3&sortOption=price-asc-rank')


In [97]:
laptop_BF_df = pd.concat([amazon_df1, amazon_df2, amazon_df3], axis=0, ignore_index=True)
laptop_BF_df['Department'] = 'Laptops'

In [98]:
laptop_BF_df.to_csv("laptops_amazon_data.csv", header=True, index=True)

In [149]:
laptop_BF_df

,title,link,manifacture,price,discount,availability,rating,number_of_rating,description,Department
0,"Lenovo TB-X306 LPDDR4X 2GB, 32GB (ZA6V0038EG)",https://www.amazon.eg/-/en/Lenovo-TB-X306-LPDDR4X-2GB-32GB-ZA6V0038EG/dp/B0BH4ZBC8P?ref_=Oct_DLandingS_D_912cee26_0,Lenovo,"4,460.",-16%,In Stock,5.0 out of 5 stars,1 rating,Brand Name Lenovo Model Name TB-X306 Screen Size 10.1 Inches Color Gray Hard Disk Size 32 GB RAM Memory Installed 2 GB Operating System Android Graphics Description Integrated Resolution 1280 x 800 Item Weight 420 Grams,Laptops
1,"itel 15.6 inch,FHD,Core-i3 1115G4,4GB+256GB SSD,50Wh, Dos",https://www.amazon.eg/-/en/itel-15-6-Core-i3-1115G4-256GB/dp/B0BSLP4X6T?ref_=Oct_DLandingS_D_912cee26_1,Itel,"10,510.",,In Stock,Previous page,0 rates,Brand Name Itel Model Name itel 15 Screen Size 15.6 Inches Color Silver Hard Disk Size 256 GB CPU Model Core i3-9300T RAM Memory Installed 8 GB Operating System Windows 10 Graphics Description Integrated CPU Speed 3 GHz,Laptops
2,"Lenovo V15 Laptop - Intel Core i3-10110U, 8GB RAM, 1TB HDD, Integrated Intel UHD Graphics, 15.6"" FHD (1920x1080) TN 220nits Anti-glare, Dos - Iron Grey",https://www.amazon.eg/-/en/Lenovo-V15-Laptop-Integrated-Anti-glare/dp/B0C2GL2W8J?ref_=Oct_DLandingS_D_912cee26_2,LENOVO,"11,680.",-9%,In Stock,Previous page,0 rates,Brand Name LENOVO Model Name V15 - I3-10110U8G1TFHD Screen Size 15.6 Inches Color Grey Hard Disk Size 1 TB CPU Model Intel Core i3 RAM Memory Installed 8 GB Operating System DOS Special Features Anti Glare Screen Graphics Description Integrated,Laptops
3,"Lenovo V15 Laptop - Intel Core i3-10110U, 4GB RAM, 1TB HDD, Integrated Intel UHD Graphics, 15.6"" HD (1366x768) TN 220nits Anti-glare, Dos - Iron Grey",https://www.amazon.eg/-/en/Lenovo-V15-Laptop-Integrated-Anti-glare/dp/B0C2GF8C51?ref_=Oct_DLandingS_D_912cee26_3,LENOVO,"12,320.",,In Stock,Previous page,0 rates,Brand Name LENOVO Model Name V15 - 82NB0015ED Screen Size 15.6 Inches Color Grey Hard Disk Size 1 TB CPU Model Intel Core i3 RAM Memory Installed 4 GB Operating System DOS Special Features Anti Glare Screen Graphics Description Integrated,Laptops
4,"Dell Vostro 3510 laptop - 11th Intel core i3-1115G4, 12 GB RAM, 256 GB SSD, Intel UHD Graphics, 15.6"" HD (1366X768) Anti-glare - Black",https://www.amazon.eg/-/en/Dell-Vostro-3510-laptop-Anti-glare/dp/B0BTGWCLXY?ref_=Oct_DLandingS_D_912cee26_4,DELL,"13,760.",-10%,In Stock,3.0 out of 5 stars,7 ratings,Brand Name DELL Model Name Vostro 3510 i3 12G 256 ub Screen Size 15.6 Inches Color Black Hard Disk Size 256 CPU Model Intel Core i3 RAM Memory Installed 12 GB Special Features Anti Glare Screen Graphics Description Integrated Graphics Coprocessor Intel UHD Graphics,Laptops
5,"Lenovo V15 G2 Laptop - Ryzen 5 5500U 6-Cores, 8 GB RAM, 512 GB SSD PCIe NVMe, Integrated AMD Radeon Graphics, 15.6"" FHD (1920x1080) TN 250nits Anti-glare, Dos - Black",https://www.amazon.eg/-/en/Lenovo-V15-Laptop-Integrated-Anti-glare/dp/B0CC1NJRZZ?ref_=Oct_DLandingS_D_912cee26_5,Lenovo,"16,968.",,In Stock,Previous page,0 rates,Brand Name Lenovo Model Name Lenovo V15 G2 5500U 8G 512G Screen Size 15.6 Inches Color Black Hard Disk Size 512 GB CPU Model AMD Ryzen 5 5500U RAM Memory Installed 8 GB Operating System DOS Special Features Numeric Keypad Graphics Description Integrated,Laptops
6,"Lenovo IdeaPad 3 15ALC6 Laptop, R5-5500U, 8GB, 512GB SSD, AMD Graphics, 15.6 FHD, DOS, Abyss Blue",https://www.amazon.eg/-/en/Lenovo-IdeaPad-15ALC6-R5-5500U-Graphics/dp/B0BS149BKH?ref_=Oct_DLandingS_D_912cee26_6,Lenovo,"18,490.",,Not Available,Previous page,0 rates,Brand Name Lenovo Model Name IdeaPad 3 15ALC6 Screen Size 15.6 Inches Hard Disk Size 512 GB CPU Model AMD Ryzen 5 5500U RAM Memory Installed 8 GB Operating System DOS Special Features Anti Glare Screen Graphics Description Integrated CPU Speed 4 GHz,Laptops
7,"Lenovo IdeaPad 3 15ALC6 Laptop - Ryzen 7 5700U 8 Cores, 8GB RAM, 512 GB PCIe NVMe, Integrated AMD Radeon Graphics, 15.6"" FHD (1920x1080) TN 250nits Anti-glare, Do

# the reviews

In [33]:
# Header to set the requests as a browser requests
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9,bn;q=0.8',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Google Chrome";v="102"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}


In [36]:
# Extra Data as Html object from amazon Review page
def reviewsHtml(url, len_page):
    
    # Empty List define to store all pages html data
    soups = []
    
    # Loop for gather all 3000 reviews from 300 pages via range
    for page_no in range(1, len_page + 1):
        
        # parameter set as page no to the requests body
        params = {
            'ie': 'UTF8',
            'reviewerType': 'all_reviews',
            'filterByStar': 'critical',
            'pageNumber': page_no,
        }
        
        # Request make for each page
        response = requests.get(url, headers=headers)
        
        # Save Html object by using BeautifulSoup4 and lxml parser
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Add single Html page data in master soups list
        soups.append(soup)
        
    return soups


In [43]:
# Grab Reviews name, description, date, stars, title from HTML
def getReviews(html_data):

    # Create Empty list to Hold all data
    data_dicts = []
    
    # Select all Reviews BOX html using css selector
    boxes = html_data.select('div[data-hook="review"]')
    
    # Iterate all Reviews BOX 
    for box in boxes:
        
        # Select Name using css selector and cleaning text using strip()
        # If Value is empty define value with 'N/A' for all.
        try:
            name = box.select_one('[class="a-profile-name"]').text.strip()
        except Exception as e:
            name = 'N/A'

        try:
            stars = box.select_one('[data-hook="review-star-rating"]').text.strip().split(' out')[0]
        except Exception as e:
            stars = 'N/A'   

        try:
            title = box.select_one('[data-hook="review-title"]').text.strip()
        except Exception as e:
            title = 'N/A'

        try:
            # Convert date str to dd/mm/yyy format
            datetime_str = box.select_one('[data-hook="review-date"]').text.strip().split(' on ')[-1]
            date = datetime.strptime(datetime_str, '%B %d, %Y').strftime("%d/%m/%Y")
        except Exception as e:
            date = 'N/A'

        try:
            description = box.select_one('[data-hook="review-body"]').text.strip()
        except Exception as e:
            description = 'N/A'

        # create Dictionary with al review data 
        data_dict = {
            'Name' : name,
            'Stars' : stars,
            'Title' : title,
            'Description' : description
        }

        # Add Dictionary in master empty List
        data_dicts.append(data_dict)
    
    return data_dicts

In [124]:
def reviews(len_page,reviews_url,index):
    # Grab all HTML
    
    html_datas = reviewsHtml(reviews_url, len_page)
    
    # Empty List to Hold all reviews data
    
    reviews = []
    # Iterate all Html page 
    for html_data in html_datas:
    
    # Grab review data
        review = getReviews(html_data)
    
    # add review data in reviews empty list
        reviews += review
        # Create a dataframe with reviews Data
    df_reviews = pd.DataFrame(reviews)
    df_reviews['index'] = index
    return df_reviews

In [161]:
laptop_BF_df

,title,link,manifacture,price,discount,availability,rating,number_of_rating,description,Department
0,"Lenovo TB-X306 LPDDR4X 2GB, 32GB (ZA6V0038EG)",https://www.amazon.eg/-/en/Lenovo-TB-X306-LPDDR4X-2GB-32GB-ZA6V0038EG/dp/B0BH4ZBC8P?ref_=Oct_DLandingS_D_912cee26_0,Lenovo,"4,460.",-16%,In Stock,5.0 out of 5 stars,1 rating,Brand Name Lenovo Model Name TB-X306 Screen Size 10.1 Inches Color Gray Hard Disk Size 32 GB RAM Memory Installed 2 GB Operating System Android Graphics Description Integrated Resolution 1280 x 800 Item Weight 420 Grams,Laptops
1,"itel 15.6 inch,FHD,Core-i3 1115G4,4GB+256GB SSD,50Wh, Dos",https://www.amazon.eg/-/en/itel-15-6-Core-i3-1115G4-256GB/dp/B0BSLP4X6T?ref_=Oct_DLandingS_D_912cee26_1,Itel,"10,510.",,In Stock,Previous page,0 rates,Brand Name Itel Model Name itel 15 Screen Size 15.6 Inches Color Silver Hard Disk Size 256 GB CPU Model Core i3-9300T RAM Memory Installed 8 GB Operating System Windows 10 Graphics Description Integrated CPU Speed 3 GHz,Laptops
2,"Lenovo V15 Laptop - Intel Core i3-10110U, 8GB RAM, 1TB HDD, Integrated Intel UHD Graphics, 15.6"" FHD (1920x1080) TN 220nits Anti-glare, Dos - Iron Grey",https://www.amazon.eg/-/en/Lenovo-V15-Laptop-Integrated-Anti-glare/dp/B0C2GL2W8J?ref_=Oct_DLandingS_D_912cee26_2,LENOVO,"11,680.",-9%,In Stock,Previous page,0 rates,Brand Name LENOVO Model Name V15 - I3-10110U8G1TFHD Screen Size 15.6 Inches Color Grey Hard Disk Size 1 TB CPU Model Intel Core i3 RAM Memory Installed 8 GB Operating System DOS Special Features Anti Glare Screen Graphics Description Integrated,Laptops
3,"Lenovo V15 Laptop - Intel Core i3-10110U, 4GB RAM, 1TB HDD, Integrated Intel UHD Graphics, 15.6"" HD (1366x768) TN 220nits Anti-glare, Dos - Iron Grey",https://www.amazon.eg/-/en/Lenovo-V15-Laptop-Integrated-Anti-glare/dp/B0C2GF8C51?ref_=Oct_DLandingS_D_912cee26_3,LENOVO,"12,320.",,In Stock,Previous page,0 rates,Brand Name LENOVO Model Name V15 - 82NB0015ED Screen Size 15.6 Inches Color Grey Hard Disk Size 1 TB CPU Model Intel Core i3 RAM Memory Installed 4 GB Operating System DOS Special Features Anti Glare Screen Graphics Description Integrated,Laptops
4,"Dell Vostro 3510 laptop - 11th Intel core i3-1115G4, 12 GB RAM, 256 GB SSD, Intel UHD Graphics, 15.6"" HD (1366X768) Anti-glare - Black",https://www.amazon.eg/-/en/Dell-Vostro-3510-laptop-Anti-glare/dp/B0BTGWCLXY?ref_=Oct_DLandingS_D_912cee26_4,DELL,"13,760.",-10%,In Stock,3.0 out of 5 stars,7 ratings,Brand Name DELL Model Name Vostro 3510 i3 12G 256 ub Screen Size 15.6 Inches Color Black Hard Disk Size 256 CPU Model Intel Core i3 RAM Memory Installed 12 GB Special Features Anti Glare Screen Graphics Description Integrated Graphics Coprocessor Intel UHD Graphics,Laptops
5,"Lenovo V15 G2 Laptop - Ryzen 5 5500U 6-Cores, 8 GB RAM, 512 GB SSD PCIe NVMe, Integrated AMD Radeon Graphics, 15.6"" FHD (1920x1080) TN 250nits Anti-glare, Dos - Black",https://www.amazon.eg/-/en/Lenovo-V15-Laptop-Integrated-Anti-glare/dp/B0CC1NJRZZ?ref_=Oct_DLandingS_D_912cee26_5,Lenovo,"16,968.",,In Stock,Previous page,0 rates,Brand Name Lenovo Model Name Lenovo V15 G2 5500U 8G 512G Screen Size 15.6 Inches Color Black Hard Disk Size 512 GB CPU Model AMD Ryzen 5 5500U RAM Memory Installed 8 GB Operating System DOS Special Features Numeric Keypad Graphics Description Integrated,Laptops
6,"Lenovo IdeaPad 3 15ALC6 Laptop, R5-5500U, 8GB, 512GB SSD, AMD Graphics, 15.6 FHD, DOS, Abyss Blue",https://www.amazon.eg/-/en/Lenovo-IdeaPad-15ALC6-R5-5500U-Graphics/dp/B0BS149BKH?ref_=Oct_DLandingS_D_912cee26_6,Lenovo,"18,490.",,Not Available,Previous page,0 rates,Brand Name Lenovo Model Name IdeaPad 3 15ALC6 Screen Size 15.6 Inches Hard Disk Size 512 GB CPU Model AMD Ryzen 5 5500U RAM Memory Installed 8 GB Operating System DOS Special Features Anti Glare Screen Graphics Description Integrated CPU Speed 4 GHz,Laptops
7,"Lenovo IdeaPad 3 15ALC6 Laptop - Ryzen 7 5700U 8 Cores, 8GB RAM, 512 GB PCIe NVMe, Integrated AMD Radeon Graphics, 15.6"" FHD (1920x1080) TN 250nits Anti-glare, Do

In [249]:
reviews1 = reviews(1,'https://www.amazon.eg/-/en/Dell-Vostro-3510-laptop-Anti-glare/product-reviews/B0BTGWCLXY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',4)


In [250]:
reviews1

,Name,Stars,Title,Description,index
0,Hisham Samir,3.0,3.0 out of 5 stars\nWarranty,Warranty for only 6 months and the vendor don’t provide an invoice.,4
1,Ahmed Elshafey,5.0,5.0 out of 5 stars\nجاء بنسخه اوبنتو وليس ويندوز,لاب جيد للأعمال المكتبيه,4
2,user-MEXKC31,1.0,1.0 out of 5 stars\nDoes not come with Windows 10 Pro as advertised,The laptop doesn’t come with Operating System: Windows 10 Pro as advertised. It came with Ubuntu operating system.,4
3,yousef badawi mostafa,3.0,3.0 out of 5 stars\nهل الجهاز جديد بكرتونتة ولا استيراد,محتاج اعرف الجهاز جديد ولا استيراد,4
4,هل المنتج جديد ولوحة المفاتيح حروف عربي,1.0,1.0 out of 5 stars\nهل المنتج جديد. لوحة المفاتيح حروف عربي,لسة مشترتوش,4


In [251]:
reviews2 = reviews(1,'https://www.amazon.eg/-/en/Dell-Vostro-3510-laptop-Anti-glare/product-reviews/B0BTGWCLXY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',8)

In [252]:
reviews2

,Name,Stars,Title,Description,index
0,Hisham Samir,3.0,3.0 out of 5 stars\nWarranty,Warranty for only 6 months and the vendor don’t provide an invoice.,8
1,Ahmed Elshafey,5.0,5.0 out of 5 stars\nجاء بنسخه اوبنتو وليس ويندوز,لاب جيد للأعمال المكتبيه,8
2,user-MEXKC31,1.0,1.0 out of 5 stars\nDoes not come with Windows 10 Pro as advertised,The laptop doesn’t come with Operating System: Windows 10 Pro as advertised. It came with Ubuntu operating system.,8
3,yousef badawi mostafa,3.0,3.0 out of 5 stars\nهل الجهاز جديد بكرتونتة ولا استيراد,محتاج اعرف الجهاز جديد ولا استيراد,8
4,هل المنتج جديد ولوحة المفاتيح حروف عربي,1.0,1.0 out of 5 stars\nهل المنتج جديد. لوحة المفاتيح حروف عربي,لسة مشترتوش,8


In [253]:
reviews3  = reviews(1,'https://www.amazon.eg/-/en/ASPIRE5-RYZEN7-5700U-RADEON-RX640-15-6FHD/product-reviews/B0C43P57VC/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',9)

In [254]:
reviews4  = reviews(1,'https://www.amazon.eg/-/en/Laptop-X515EP-BQ8G5W-Generation-Intel%C2%AE-i5-1135G7/product-reviews/B0BR4G9Q27/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',12)

In [255]:
reviews4

,Name,Stars,Title,Description,index
0,Menem sayed,3.0,3.0 out of 5 stars\nلابتوب مكتب جميل,لابتوب بإمكانيات رائعة لشغل المكتب و شيك لكنه ليس للالعاب والجرافيك,12
1,Abdellatif,5.0,5.0 out of 5 stars\nجهاز رائع من اسوس,Slim device with strong precise ports. The display is FHD. The basic specs are as described. i5-1135G7 is showing high performance.,12
2,Mohamed Ayman,5.0,5.0 out of 5 stars\nبراند ممتاز جدا انصح به,اللاب ما شاء الله جميل جدا كتصميم و خامات و الكيبورد اضاءه ابيض 3 مستويات ، كما توقعت و اكثر اللاب مش هيقصر معاك حتى في لعبه GTA V شغاله كويس جدا بس هتقلل الجوده شويه ده غير ضمان اسوس المميز سنتين كاملين و تم اضافه الفاتوره الضريبيه من قبل امازون بعد 3 ايام ، شكرا امازون ❤,12
3,adrianwassim,5.0,5.0 out of 5 stars\ngood laptop for price,good laptop for price but doesn't have 2nd authentication like fingerprint or face recognition,12


In [256]:
reviews5  = reviews(1,'https://www.amazon.eg/-/en/Asus-Vivobook-D1603QA-MB007W-Ryzen7-5800H/product-reviews/B0C4JXXDKK/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',12)
reviews5

,Name,Stars,Title,Description,index
0,JUAN CARLOS,N/A,Pesima laptop,"Se descompuso a los 6 meses de uso, no la compres",12
1,Laura Rmz,N/A,Pésimo,"Pésimo producto, lento al iniciar, lento al trabajar, más de 10min en iniciar sesión ya no alcanzamos garantía porque quien la usa es un familiar, yo soy la que revisa los equipos.",12
2,Magdishart,N/A,Excelente,"Excelente computadora, sin duda lo que esperaba",12


In [257]:
reviews6  = reviews(1,'https://www.amazon.eg/-/en/Dell-Vostro-3510-laptop-Anti-glare/product-reviews/B0BNKCW59F/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',13)
reviews6

,Name,Stars,Title,Description,index
0,Mohamed wael,5.0,5.0 out of 5 stars\nReally awesome,The laptop is really awesome came as described. Fast and does all what works needs I didn’t find any laptop can beat this one with same price or qualifications for sure it has a better graphics card it would be fun for gaming too but that’s another story . I really recommended,13


In [258]:
reviews7  = reviews(1,'https://www.amazon.eg/-/en/Lenovo-IdeaPad-Laptop-i7-1165G7-1920x1080/product-reviews/B0BBSQPWV9/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',16)
reviews7

,Name,Stars,Title,Description,index
0,Mohamed Qenawy,5.0,5.0 out of 5 stars\nجهاز جيد ومطابق للمواصفات,اعجبنى فى الجهاز شكله الجميل،وخفة وزنه الى جانب كونه فى حالة جديدة وممتازة، فقط يجب مراعاة ان الجهاز لا يوجد عليه نظام تشغيل، وتم انزال نسخة من نظام التشغيل عليه ويعمل بصورة رائعة، وبالتالى يجب تعديل المواصفات على صفحة اعلان الجهاز وازالة كلمة نظام تشغيل ويندوز 10 حتى لا يحدث التباس لدى المشترى، وافر الشكر,16


In [259]:
reviews8  = reviews(1,'https://www.amazon.eg/-/en/Lenovo-IdeaPad-i7-1165G7-Fingerprint-Backlight/product-reviews/B09CLYRF1L/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',18)
reviews8

,Name,Stars,Title,Description,index
0,haithamhcai,4.0,4.0 out of 5 stars\nجيد جدا,جيد جدا,18
1,mselmadawy,5.0,5.0 out of 5 stars\nVery good,Very good,18
2,hisham ibrahim,5.0,5.0 out of 5 stars\nلابتوب ممتاز,سعره و بطاريه و خفه وزن ممتازالشاشه الونها نص نص,18


In [260]:
reviews9  = reviews(1,'https://www.amazon.eg/-/en/Lenovo-IdeaPad-15ACH6-Rayzen5-NV-RTX3050TI/product-reviews/B0BKQW8WN5/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',20)
reviews9

,Name,Stars,Title,Description,index
0,Dave Garcia,N/A,"Worthwhile bargain laptop if capable of DIY upgrades, watch for refurbs if that matters","Bought this product on sale as a gift for about 650 total. Received it when expected but in packaging that was truly beat to hell. The box was covered in 2 different types of tapes so it’s also entirely possible that with all the craziness of Black Friday the outside-most box just got decimated. No star loss here as the inside box had the appropriate amount of shock resistant packaging material to make it seem protected despite circumstances. I opened the undamaged computer and moved onto the required ssd and ram upgrades and the optional battery upgrade.I wanted to point out that through the Lenovo Vantage app I found out that the battery warranty had already started back in August. From my experience with the Lenovo Legion 5 that period starts once someone registers the product. After searching there is a place to note the date as incorrect and resubmit through the email created alongside the Microsoft one during setup- for people uncomfortable with products that appear to be returned/refurbished be forewarned that appears to be potentially possible. In my case it wasn’t very relevant as you will see. After a more careful inspection I moved on, but nothing in the product description, listing, or packaging made it seem like anything but new. Skip to the end but if you want my attempt at tldr.The required ssd upgrade was a 500gb 2280 NVMe Samsung 970 Evo Plus, the ram was recycled from the above mentioned legion 5 but they are two 8gbs running at 3200m/t from Critical, and optionally I decided to try to port the newish 60wh battery from the legion 5 to replace the 45wh within. The ram and ssd were typical, the laptop came with a baggie of screws that sorta fit the 2280 screw hole but left just a bit of play that does long term worry me will potentially loosen. The battery upgrade did require relocating the bios battery 2/3 inches and the larger battery fit but required some firm but gentle coaxing to sink in its new home. Case closed normally and battery was recognized in Lenovo vantage with normal temps.All upgrades were recognized in the system and after tweaks, updates, and stuff- the attached photo shows the score in user benchmark. If you time the deals correctly the ram and ssd upgrades should be sub 150- even sub 100 if you really got a tight budget. The 60 wh battery can be found 3rd party for a reasonable price as well and with necessary connections, but if you’ll only ever use it plugged in that might not matter. Overall, for the price this laptop can’t be beat as a sub 1k gaming laptop. L4D2 with maxed settings and graphically intense workshop content added barely touches 45% gpu load per Nvidia’s stats. Newer content such as Deep Rock Galactic ran closer to 80-85 fps on maxed settings so playing around with the options/DLSS/reflex boost can yield close to or in excess of the 120hz refresh rate if that matters as well. Overall, 1080p gaming is probably no worries but I didn’t test Crysis so who knows.***The most important thing to say imo is that if you’re unwilling or unable to do the upgrades then I would avoid this computer at any price point.*** The Ryzen 5 gpu hardware reserves 2g of ram by default (disabling the integrated gpu does not yield worthwhile results- don’t bother) and lowering the UMA to 512mb costs more in the computers ability to multitask/multimedia surf effectively. User benchmark proved the performance was worse. That left the free memory on the computer at around 2-3gbs for gaming on the base version which would bottleneck very quickly. Furthermore you definitely want to move windows to whatever 2280 NVMe you upgrade with, the base 256 gb 2242 (or 2246 i forget the size) is adequate but uninspiring and you would benefit from the main system being on the higher bandwidth option- leaving the base storage as overflow.For the DIY capable or willing, this 

In [261]:
reviews10  = reviews(3,'https://www.amazon.eg/-/en/Lenovo-Legion-15ACH6-Gaming-Laptop/product-reviews/B0BNZCHJMW/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3',25)
reviews10

,Name,Stars,Title,Description,index
0,Ricky,N/A,Great Price & Great Laptop,Very happy with my laptop purchase. Shipped quickly and everything was in order and not missing/damaged.,25
1,Javier Garcia,N/A,Una buena laptop,Una buena laptop calidad precio y el vendedor la entrego antes de lo esperado,25
2,Alejandro,N/A,Defecto en la pantalla,Compré este modelo y la pantalla tiene muchas fugaz de luz. Demasiadas termina arruinando la experiencia por completo.He tratado de contactar al proveedor pero se tardan demasiado en responder y no parece existir un en ayudarme.La verdad no creo que sea un modelo tan barato como para no recibir una buena atención.,25
3,Rene Hernandez,N/A,Got an opened box item not a new item.,The security seals were obviously tampered with when i got the package. One security seal was missing and replaced with packing tape,25
4,Ricky,N/A,Great Price & Great Laptop,Very happy with my laptop purchase. Shipped quickly and everything was in order and not missing/damaged.,25
5,Javier Garcia,N/A,Una buena laptop,Una buena laptop calidad precio y el vendedor la entrego antes de lo esperado,25
6,Alejandro,N/A,Defecto en la pantalla,Compré este modelo y la pantalla tiene muchas fugaz de luz. Demasiadas termina arruinando la experiencia por completo.He tratado de contactar al proveedor pero se tardan demasiado en responder y no parece existir un en ayudarme.La verdad no creo que sea un modelo tan barato como para no recibir una buena atención.,25
7,Rene Hernandez,N/A,Got an opened box item not a new item.,The security seals were obviously tampered with when i got the package. One security seal was missing and replaced with packing tape,25
8,Ricky,N/A,Great Price & Great Laptop,Very happy with my laptop purchase. Shipped quickly and everything was in order and not missing/damaged.,25
9,Javier Garcia,N/A,Una buena laptop,Una buena laptop calidad precio y el vendedor la entrego antes de lo esperado,25


In [262]:
reviews11  = reviews(1,'https://www.amazon.eg/-/en/HP-Generation-HardDisk-Graphic-4R305T/product-reviews/B09VZKKMKT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',26)

In [263]:
reviews12  = reviews(1,'https://www.amazon.eg/-/en/Lenovo-ThinkPad-Yoga-Gen2-Laptop/product-reviews/B0C3V8XYQR/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',27)

In [264]:
reviews13  = reviews(1,'https://www.amazon.eg/-/en/Dell-G15-5511-Gaming-Laptop/product-reviews/B0BJNYMT9S/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',34)

In [265]:
reviews14  = reviews(2,'https://www.amazon.eg/-/en/Dell-G15-5511-Gaming-Laptop-i7-11800H/product-reviews/B09QPBLGR5/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2',38)

In [266]:
reviews15  = reviews(3,'https://www.amazon.eg/-/en/Lenovo-IdeaPad-Gaming-15ACH6-Laptop/product-reviews/B0BGLN9JKS/ref=cm_cr_getr_d_paging_btm_next_3?ie=UTF8&reviewerType=all_reviews&pageNumber=3',39)

In [267]:
reviews16  = reviews(1,'https://www.amazon.eg/-/en/Dell-Gaming-15-6-inch-Non-Touch-Laptop/product-reviews/B0979GB22W/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',42)

In [172]:
reviews16

,Name,Stars,Title,Description,index
0,Hip-Hop Anonymouse,N/A,"For the price it's a good deal, but you could do much better...",An RTX 3060 laptop for under $1400 (with $140 discount at the time of purchase) seemed like a good deal but to get to that price point a lot was sacrificed it seems.Pros:- Price was affordable- Build quality is very sturdy. Very little flex in the chassis or screen.- In-stock. Nearly everything else I wanted was out of stock or price gouged by scalpers- No dead pixels as far as I can tell.- Nice speakers but fans can still be heard over them.Cons:- TDP of 3060 is only 90 watts. Confirmed with GPU-Z.- Gets VERY Hot. GPU easily hit 90 degrees Celsius then crashed while gaming from overheating.- Keyboard is not great. Missed key presses constantly and using red letters on black keys is a terrible idea.- Mux switch is in the BIOS and can not be changed in Windows- Bloatware. 512gb SSD but only 407gb free.- Some backlight bleed. Very noticeable on dark scenes.- Ships with dead battery. I thought I had a DOA unit cause it would not boot up.- Color is not black. It's a very dark dirty grey unlike the pictures.Overall its a nice laptop but I was mislead by reviews online and and the Q&A section that this was the high wattage 3060. Due to the thermal issue I'm guessing that's why they opted for the lower watt version. The cooling could be better IMHO as evidenced by the crashes I experience from overheating. If you can get it on a sale and not MSRP its a better deal than most 3050 ti laptops. It's good enough for basic gaming but don't expect to run modern games on it at high framerates/quality. Despite having a Core i7 CPU I'm constantly experiencing system stuttering which could be a windows issue or caused by all the bloatware. Dell did not include any backup disks or a restore option so I can not uninstall anything without losing it forever. I will update my review after the return period is over to see if it holds up.,42
1,Brisur,N/A,Defectuosa después de un año,"A los seis meses falló la batería, se apagó y no volvió a encender. La llevé a la garantía y se la cambiaron. Siete meses después de eso, volvió a apagarse. Ya no entró en garantía, y resulta que es la tarjeta madre >:(. Pésima calidad de Dell.Además, en el centro de servicio Dell en Yucatán, siempre tienen de la misma línea, por que están falladas de lo mismo: se apagan y no vuelven a encender. Venden porquerías carísimas >:(",42
2,Ben,N/A,its great,had it for about a month now issues at allgame mode is great and doesnt get too hot on your lap,42
3,Soltan,N/A,Good performance for a good price,In use already 2 weeks. Satisfied with everything,42


In [268]:
reviews17  = reviews(1,'https://www.amazon.eg/-/en/FX517ZM-HF083W-Generation-Intel%C2%AE-CoreTM-I7-12650H/product-reviews/B0BNR2LR2Z/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',47)

In [171]:
reviews17

,Name,Stars,Title,Description,index
0,Rami ghonima,5.0,5.0 out of 5 stars\nمنتجات Asus ممتازة,لاب توب قوي جداً، انصح اي حد بشرائه لما لان امكانياته عالية جداً و شركة اسوس من احسن الشركات اللي تعاملت معاها عن تجربة و كمان الضمان بتاعهم وخدمة ما بعد البيع ممتازة يعني لو احتاجت اي حاجة بتلاقي الدعم المطلوب منهم.و طبعاً الضمان بتاعهم دولي مش محلي فقط و دا شئ عظيم جداً.شكراً اسوس,47
1,John,5.0,5.0 out of 5 stars\nGreat laptob,"It's a very good and very impressive laptop, it's suitable for everything and everyday work : (gaming/ movies/ rendering) and also its not very heavy like any other gaming laptop which is very good thingAnd it's battery also good not very bad like any gaming laptopAnd also the display colors not the best in the market but you have to remember that's a budget laptopI gamed on it and done some rendering stuffOverall its great laptob and great value for money",47


In [269]:
reviews18  = reviews(2,'https://www.amazon.eg/-/en/ASUS-G533ZX-LN060W-I9-12900H-RTX3080Ti-Warranty/product-reviews/B0BGL51D92/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=2',57)

In [270]:
reviews18

,Name,Stars,Title,Description,index
0,TAGZO,N/A,Its to brag about,"It is one of the fastest, if not the fastest, laptops you can buy right now!",57
1,Daniela Cunha,N/A,Runs pretty much every game,One of the best gaming laptops I ever tried. Worth every price. Fasting shipping. Came perfectly easy to set up! Overall very pleased with it,57
2,Leo,N/A,Industrial waste,Industrial waste. 3d mark can't run,57
3,TAGZO,N/A,Its to brag about,"It is one of the fastest, if not the fastest, laptops you can buy right now!",57
4,Daniela Cunha,N/A,Runs pretty much every game,One of the best gaming laptops I ever tried. Worth every price. Fasting shipping. Came perfectly easy to set up! Overall very pleased with it,57
5,Leo,N/A,Industrial waste,Industrial waste. 3d mark can't run,57


In [271]:
all_reviews = pd.concat([reviews1, reviews2,reviews3,reviews4,reviews5,reviews6,reviews7,reviews8,reviews9,reviews10,reviews11,reviews12,reviews13,reviews14,reviews15,reviews16,reviews17,reviews18], axis=0, ignore_index=True)

In [272]:
all_reviews

,Name,Stars,Title,Description,index
0,Hisham Samir,3.0,3.0 out of 5 stars\nWarranty,Warranty for only 6 months and the vendor don’t provide an invoice.,4
1,Ahmed Elshafey,5.0,5.0 out of 5 stars\nجاء بنسخه اوبنتو وليس ويندوز,لاب جيد للأعمال المكتبيه,4
2,user-MEXKC31,1.0,1.0 out of 5 stars\nDoes not come with Windows 10 Pro as advertised,The laptop doesn’t come with Operating System: Windows 10 Pro as advertised. It came with Ubuntu operating system.,4
3,yousef badawi mostafa,3.0,3.0 out of 5 stars\nهل الجهاز جديد بكرتونتة ولا استيراد,محتاج اعرف الجهاز جديد ولا استيراد,4
4,هل المنتج جديد ولوحة المفاتيح حروف عربي,1.0,1.0 out of 5 stars\nهل المنتج جديد. لوحة المفاتيح حروف عربي,لسة مشترتوش,4
...,...,...,...,...,...
106,Daniela Cunha,N/A,Runs pretty much every game,One of the best gaming laptops I ever tried. Worth every price. Fasting shipping. Came perfectly easy to set up! Overall very pleased with it,57
107,Leo,N/A,Industrial waste,Industrial waste. 3d mark can't run,57
108,TAGZO,N/A,Its to brag about,"It is one of the fastest, if not the fastest, laptops you can buy right now!",57
109,Daniela Cunha,N/A,Runs pretty much every game,One of the best gaming laptops I ever tried. Worth every price. Fasting shipping. Came perfectly easy to set up! Overall very pleased with it,57


In [273]:
all_reviews.to_csv("all_reviews.csv", header=True, index=True)

In [276]:
pd.read_csv('all_reviews.csv').drop(columns=['Unnamed: 0'])

,Name,Stars,Title,Description,index
0,Hisham Samir,3.0,3.0 out of 5 stars\nWarranty,Warranty for only 6 months and the vendor don’t provide an invoice.,4
1,Ahmed Elshafey,5.0,5.0 out of 5 stars\nجاء بنسخه اوبنتو وليس ويندوز,لاب جيد للأعمال المكتبيه,4
2,user-MEXKC31,1.0,1.0 out of 5 stars\nDoes not come with Windows 10 Pro as advertised,The laptop doesn’t come with Operating System: Windows 10 Pro as advertised. It came with Ubuntu operating system.,4
3,yousef badawi mostafa,3.0,3.0 out of 5 stars\nهل الجهاز جديد بكرتونتة ولا استيراد,محتاج اعرف الجهاز جديد ولا استيراد,4
4,هل المنتج جديد ولوحة المفاتيح حروف عربي,1.0,1.0 out of 5 stars\nهل المنتج جديد. لوحة المفاتيح حروف عربي,لسة مشترتوش,4
...,...,...,...,...,...
106,Daniela Cunha,NaN,Runs pretty much every game,One of the best gaming laptops I ever tried. Worth every price. Fasting shipping. Came perfectly easy to set up! Overall very pleased with it,57
107,Leo,NaN,Industrial waste,Industrial waste. 3d mark can't run,57
108,TAGZO,NaN,Its to brag about,"It is one of the fastest, if not the fastest, laptops you can buy right now!",57
109,Daniela Cunha,NaN,Runs pretty much every game,One of the best gaming laptops I ever tried. Worth every price. Fasting shipping. Came perfectly easy to set up! Overall very pleased with it,57


In [278]:
data_generate('https://www.amazon.eg/s?rh=n%3A21833043031%2Cp_72%3A4-&language=en&content-id=amzn1.sym.e28a1dba-db01-4b13-98a5-736a0b8c120b&pd_rd_r=4ec00558-dda5-42b1-a0b8-abd74163bebf&pd_rd_w=vRTcX&pd_rd_wg=Zf9fj&pf_rd_p=e28a1dba-db01-4b13-98a5-736a0b8c120b&pf_rd_r=WAXHJ26VWJMQJ1AJ1723&ref=Oct_d_otopr_S')

,title,link,manifacture,price,discount,availability,rating,number_of_rating,description
